In [67]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import json
from google.colab import files
import nltk
from nltk.tokenize import sent_tokenize
import torch

In [ ]:
nltk.download('punkt_tab')

In [69]:
tokenizer = AutoTokenizer.from_pretrained("lisaterumi/postagger-portuguese")
model = AutoModelForTokenClassification.from_pretrained("lisaterumi/postagger-portuguese")

In [70]:
with open("corpus.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [71]:
dados = {
    "conferencia_principal": [
    ]
    }

conferencia_principal = data["conferencia_principal"]

In [ ]:
pos_tag = pipeline("token-classification", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

In [73]:
def processar_frase(frase, tokenizer, pos_tag, max_length=510):
    tokens = tokenizer.tokenize(frase)
    resultado = []

    if len(tokens) <= max_length:
        saida = pos_tag(frase)
        resultado.append([[token["word"], token["entity_group"]] for token in saida])
    else:
        for i in range(0, len(tokens), max_length):
            tokens_chunk = tokens[i:i+max_length]
            chunk_text = tokenizer.convert_tokens_to_string(tokens_chunk)
            saida = pos_tag(chunk_text)
            resultado.append([[token["word"], token["entity_group"]] for token in saida])

    return resultado


In [ ]:
for index, artigo in enumerate(conferencia_principal):
    print(index)
    frases = sent_tokenize(artigo["artigo_completo"], language='portuguese')
    artigo["artigo_sentenças"] = frases

    pos = []
    for frase in frases:
      saida_pos = processar_frase(frase, tokenizer, pos_tag)
      pos.extend(saida_pos)

    artigo["pos_tag_artigo"] = pos
    dados["conferencia_principal"].append(artigo)

In [75]:
with open("corpus.json", "w", encoding="utf-8", ) as f:
    json.dump(dados, f, ensure_ascii=False, indent=4, default=str)

files.download("corpus.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>